## Introduction
This code defines a protocol for picking colonies from defined locations on a colony plate to a target plate for a downstream colony PCR protocol. The protocol has been split into two as the image and colony locations will have to be retreived on a seperate platform. Therefore, protocol1 must be run first so the camera can be in the correct position to take an image then once the locations are retrieved, this can be entered into the protocol2 and run to pick the colony.

### Protocol 1
The first protocol moves the robot, and hence the camera, by a calibrated offset from 'A1' of the labware below to a defined location above the colony plate. 

### Protocol 2
The second protocol takes the colony locations then moves the robot to pick the colonies and perform the rest of the colony PCR steps. The location of the colonies is defined as an offset relative to the top left corner of the light table. 

The code has been divided into sections with the functions move_camera and pick_colony for modularity thereby allowing it to be integrated into other protocols.

In [29]:
import opentrons.execute
import opentrons.simulate
import json
from opentrons.types import Location, Point
from opentrons import protocol_api

In [30]:
def move_camera(pipette, tip_box):
    '''
    This function moves the pipette, and hence the camera, to a defined location above the light table/colony plate.
    '''
    # Defines reference point to top of A1 of labware below designated light table/colony plate
    calibration_ref = tip_box['A1'].top()
    
    # Calculates coordinates of location to move to from offset
    final_offset = Point(20,-10,40) # offset from reference point
    final_position = calibration_ref.move(final_offset)
    
    #Moves camera to calculated location
    pipette.move_to(final_position)

In [31]:
# Protocol 1
metadata = {
    'apiLevel': '2.13',
    'protocolName': 'Colony Imaging'
    }
    
def run(protocol1: protocol_api.ProtocolContext):
    '''
    Protocol 1: Moves camera to defined location above the colony plate/light table
    '''
    #Homes robot
    protocol1.home()
    
    # Defines and loads labware
    tips_300 = protocol1.load_labware('opentrons_96_tiprack_300ul', 1)
    p300 = protocol1.load_instrument('p300_single_gen2', 'left', tip_racks=[tips_300])
    tips_300.set_offset(x=-0.20, y=1.30, z=-0.10)
    
    #Moves camera
    move_camera(p300, tips_300)

In [82]:
# Runs protocol 1 - MUST BE RUN BEFORE SUBSEQUENT CELLS
protocol1 = opentrons.execute.get_protocol_api('2.13')
run(protocol1)

[89.5, -47.2]
[68.5, -29.0]
[37.1, -53.2]


In [33]:
def get_colony_locations():
    '''
    Gets colony location
    '''
    #INPUT COLONY LOCATIONS HERE!!!!!!!
    colony_locations = [[0.5490428805351257,0.6883083581924438],[0.33709481358528137,0.5272704362869263],[0.6180869340896606,0.28566786646842957],[0.37698936462402344,0.3511043190956116],[0.2625761032104492,0.5011239647865295],[0.5195361375808716,0.3209857940673828],[0.5756020545959473,0.7255251407623291],[0.3273409307003021,0.49596357345581055],[0.3317597806453705,0.6452765464782715],[0.2727550268173218,0.5743262767791748],[0.5109610557556152,0.6624088287353516],[0.34613704681396484,0.6023465394973755],[0.3552238643169403,0.44675150513648987],[0.28688639402389526,0.3764688968658447],[0.5223345756530762,0.24922838807106018],[0.3631874918937683,0.4023599326610565],[0.7639864683151245,0.605918288230896],[0.3310561180114746,0.41950929164886475]]
    
    # Defines offset 
    x_offset= 0.0
    y_offset = 0.0 
    x_scaler = 130 - x_offset
    y_scaler = 86 - y_offset
    
    #Scales locations to dimensions of light table
    colony_locations = [[round(x_scaler*x[1],1), round(-y_scaler*x[0],1)] for x in colony_locations]
    return colony_locations


def pick_colony(protocol, coordinates, colony_plate, target_plate, pipette):
    '''
    This function picks the colonies after being given the colony coordinates
    '''
    # Reference point of light table/colony plate
    calibration_ref = colony_plate['A1'].top() #to be calibrated as top left corner of lightbox
    
    for coordinate in coordinates:
        col_offset = Point(x =coordinate[0], y = coordinate[1], z = 0) # colony offset on labware
        pick_offset = Point(x =coordinate[0], y = coordinate[1], z = -14) # z-axis offset to pick colony
    
        index = coordinates.index(coordinate)
        
        # Calculates absolute location of colonies
        target_location = calibration_ref.move(col_offset) # absolute coordinate of colony
        p_target_location = calibration_ref.move(pick_offset)
        
        # Pipette moves to colony x,y location
        pipette.pick_up_tip()
        pipette.move_to(target_location)
        protocol.delay(seconds=2)
        
        # Pipette moves down to pick colony
        pipette.move_to(p_target_location)
        protocol.delay(seconds=2)
        
        # Transfers colony to destination plate
        pipette.mix(3, 10, target_plate.wells(index)[0]) 
        pipette.touch_tip(target_plate.wells(index)[0], speed = 40) 
        pipette.blow_out(target_plate.wells(index)[0])
        pipette.drop_tip()

def transfer_colony_suspension(p300, sourceplate, destplate, sources:list, destinations:list):
    #Adding colony suspension to the PCR mix
    for i, source in enumerate(sources):
        p300.transfer(20, sourceplate.wells_by_name()[sources[i]], 
                      destplate.wells_by_name()[destinations[i]], new_tip = 'always')

def transfer_lb_to_suspension(p300, sourceplate, destplate, destinations:list):
    #Adding LB from the reservoir to the remaining colony suspension
    for destination in destinations: 
        p300.transfer(180, sourceplate.wells_by_name()['A1'], 
                      destplate.wells_by_name()[destination], new_tip = 'always', mix='always')
        

In [36]:
###IMPORTANT!!! - Protocol 1 must be run before this to obtain image and colony locations

metadata = {
    'apiLevel': '2.13',
    'protocolName': 'Colony Picking protocol'
    }
def run(protocol2: protocol_api.ProtocolContext):
    '''
    Protocol 2: Picks colony and runs colony PCR
    '''
    #Define labware
    tips_300 = protocol2.load_labware('opentrons_96_tiprack_300ul', 1)
    p300 = protocol2.load_instrument('p300_single_gen2', 'left', tip_racks=[tips_300])
    tips_300.set_offset(x=-0.20, y=1.30, z=-0.10) #set offset from position check
    
    lbres = protocol2.load_labware('nest_12_reservoir_15ml',3)
    destplate = protocol2.load_labware('biorad_96_wellplate_200ul_pcr', 5)
    lbplate = protocol2.load_labware('nest_96_wellplate_2ml_deep', 2)
    
    #Define light table custom labware
    with open('90mmpetridishwithlighttable_1_wellplate_250ul.json') as labware_file1:
        labware_def1 = json.load(labware_file1)
        colony_plate = protocol2.load_labware_from_definition(labware_def1, 4)
        colony_plate.set_offset(x=-12.20, y=11.70, z=2.10) #set offset from position check
    
    # Perform Colony picking and adding to the Deep well plates A2,B2 and C2
    colony_locations = get_colony_locations()
    colony_num = 3
    
    pick_colony(protocol2, colony_locations[0:colony_num], colony_plate, lbplate, p300)
    p300.home()
    
    # Execute Remaining Sub Processes - Reagents Prep
    transfer_colony_suspension(p300, lbplate, destplate, ['A1', 'B1', 'C1'], ['A1', 'B1', 'C1'])
    
    transfer_lb_to_suspension(p300, lbres, lbplate, ['A1', 'B1', 'C1'])
    

In [ ]:
#Runs protocol 2
protocol2 = opentrons.execute.get_protocol_api('2.13')
run(protocol2)

In [37]:
#Run simulation
protocol1 = opentrons.simulate.get_protocol_api('2.13')
protocol2 = opentrons.simulate.get_protocol_api('2.13')
run(protocol1)
run(protocol2)

for line in protocol1.commands(): 
    print(line)
    
for line in protocol2.commands(): 
    print(line)

C:\Users\Olaide Ibiyemi\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\Olaide Ibiyemi\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\Olaide Ibiyemi\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\Olaide Ibiyemi\.opentrons\deck_calibration.json not found. Loading defaults


Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 1
Moving to A1 of 90mm Petri Dish With Light Table 1 Well Plate 250 ÂµL on 4
Delaying for 0 minutes and 2.0 seconds
Moving to A1 of 90mm Petri Dish With Light Table 1 Well Plate 250 ÂµL on 4
Delaying for 0 minutes and 2.0 seconds
Mixing 3 times with a volume of 10.0 ul
Aspirating 10.0 uL from A1 of NEST 96 Deepwell Plate 2mL on 2 at 92.86 uL/sec
Dispensing 10.0 uL into A1 of NEST 96 Deepwell Plate 2mL on 2 at 92.86 uL/sec
Aspirating 10.0 uL from A1 of NEST 96 Deepwell Plate 2mL on 2 at 92.86 uL/sec
Dispensing 10.0 uL into A1 of NEST 96 Deepwell Plate 2mL on 2 at 92.86 uL/sec
Aspirating 10.0 uL from A1 of NEST 96 Deepwell Plate 2mL on 2 at 92.86 uL/sec
Dispensing 10.0 uL into A1 of NEST 96 Deepwell Plate 2mL on 2 at 92.86 uL/sec
Touching tip
Blowing out at A1 of NEST 96 Deepwell Plate 2mL on 2
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from B1 of Opentrons 96 Tip Rack 300 µL on 1
Moving to A1 of 90mm Petri

In [ ]:
# http://155.198.148.152:4444/pi-cam-image
# http://155.198.148.152:4444/colony-locations